<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/leerazo/neo4j_aura_api/blob/main/03-deploying_aura/03-deploying_aura.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/leerazo/neo4j_aura_api/blob/main/03-deploying_aura/03-deploying_aura.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/leerazo/neo4j_aura_api/blob/main/03-deploying_aura/03-deploying_aura.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</td>
</table>


# Deploying Neo4j using the Aura API 

Let's begin by installing all the needed libraries

In [ ]:
%pip install python-dotenv
%pip install jupyterlab

#%pip install jupyterlab_widgets
%pip install ipywidgets

In [5]:
from dotenv import dotenv_values
from urllib.parse import urljoin
import ipywidgets as widgets
from IPython.display import display
import subprocess
import datetime
import json
import os

## Step 1: Authenticate API and get a bearer token for access

Upload the API credential file you created earlier using the widget in the cell below. 

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print('Google Colab:', str(IN_COLAB))

if IN_COLAB:
    from google.colab import files
    uploaded = files.upload()
    filename = next(iter(uploaded))
    content = uploaded[filename]
    cred_file = 'aura_api_creds.txt'
    os.rename(filename, cred_file)
else:
    cred_file = ''
    print('Currently only supported in Google Colab')

Now let's extract the credentials for further use. We'll save it to a file in case we need to access it again later. 

In [ ]:
aura_creds = dotenv_values(cred_file)
for item in aura_creds:
  print(item, '=', aura_creds[item])

Let's define a helper function which uses these credentials to generate a bearer token for access.

In [ ]:
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = 'api_bearer_token'
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    return access_token 

Now let's try it.

In [ ]:
api_base = 'https://api.neo4j.io/'

access_token = refresh_token(aura_creds, api_base)
print('access_token:', access_token)


## Deploying Neo4j

First step is to choose a tenant to deploy into, let's define some helper functions.

In [ ]:
def list_tenants(access_token, api_base, tenant_id=None):
    #api_endpoint = 'https://api.neo4j.io/v1/tenants'

    api_endpoint = urljoin(api_base, '/v1/tenants')

    aura_tenants = {}

    if tenant_id:
        api_endpoint += '/' + tenant_id

    print()
    print('api_endpoint:', api_endpoint)
    print()

    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    
    list_tenants = dict(json.loads(subprocess.check_output(list_cmd, shell=True)))['data']
    for item in list_tenants:
        aura_tenants[item['id']] = {}
        aura_tenants[item['id']]['id'] = item['id'] 
        aura_tenants[item['id']]['name'] = item['name'] 

    return aura_tenants


def tenant_info(access_token, api_base, tenant_id):
    api_endpoint = urljoin(api_base, '/v1/tenants/' + tenant_id)
    tenant_data = {}
    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    tenant_info = json.loads(subprocess.check_output(info_cmd, shell=True))['data']
    
    for item in tenant_info:
        tenant_data[tenant_info['id']] = {}
        tenant_data[tenant_info['id']]['tenant_id'] = tenant_info['id']
        tenant_data[tenant_info['id']]['tenant_name'] = tenant_info['name'] 
        tenant_data[tenant_info['id']]['instance_configurations'] = list(tenant_info['instance_configurations'])

    return tenant_data


Now let's take a look at the environment, beginning with a list of tenants we have access to.

In [8]:
aura_tenants = list_tenants(access_token, api_base)
print()
print('Aura Tenants:')
print('-------------')
for tenant in aura_tenants:
    print(aura_tenants[tenant])
print()

selected_tenant = widgets.Dropdown(
    options=tenant_ids,
    #value='2',
    description='Please select a Tenant ID:',
    disabled=False,
)

display(selected_tenant)

NameError: name 'list_tenants' is not defined

Once we've chosen a tenant, copy the tenant ID.

In [ ]:
tenant_id = selected_tenant.value # Paste in the tenant_id of the tenant you wish to use 
print('tenant_id:', tenant_id)
print()

tenant_details = tenant_info(access_token, api_base, tenant_id)

print('Tenant Details:')
print('---------------')
for item in tenant_details[tenant_id]:
    print(item, '=', tenant_details[tenant_id][item])

cloud_providers = list({i['cloud_provider'] for i in tenant_details[tenant_id]['instance_configurations']})

In [ ]:
selected_csp = widgets.Dropdown(
    options=cloud_providers,
    #value='2',
    description='Please select a cloud provider:',
    disabled=False,
)

display(selected_csp)

In [ ]:
cloud_provider = selected_csp.value
print('cloud_provider:', cloud_provider)

In [ ]:
available_regions = {}
available_types = {}
available_versions = {}
available_sizes = {}
for config in instance_configurations:
   if config['cloud_provider'] == cloud_provider:
       #print(config)
       region = config['region']
       type = config['type']
       version = config['version']
       size = config['memory']
       available_regions[region] = config['region_name']
       available_types[type] = config['type']
       available_versions[version] = config['version']
       available_sizes[size] = config['memory']
region_list = list(available_regions.keys())
type_list = list(available_types.keys())
version_list = list(available_versions.keys())
size_list = list(available_sizes.keys())


selected_region = widgets.Dropdown(
    options=region_list,
    #value='2',
    description='Please select a region:',
    disabled=False,
)

selected_type = widgets.Dropdown(
    options=versions_list,
    #value='2',
    description='Please select a type:',
    disabled=False,
)

selected_version = widgets.Dropdown(
    options=version_list,
    #value='2',
    description='Please select a version:',
    disabled=False,
)

selected_size = widgets.Dropdown(
    options=size_list,
    #value='2',
    description='Please select a memory size:',
    disabled=False,
)
#display(selected_region)
display(selected_type)

In [ ]:
region = str(selected_region.value)
type = str(selected_type.value)
print('tenant_id:', tenant_id)
print('cloud_provider:', cloud_provider)
print('region:', region)
print('type:', type)

available_configs = {}

for config in instance_configurations:
    if (config['cloud_provider'] == cloud_provider) and (config['region'] == region) and (config['type'] == type):
           print(config)

Now we're ready to deploy! Let's define a couple more helper functions. 

In [ ]:
def deploy_instance(access_token, api_base, instance_name, tenant_id, cloud_provider, region='europe-west1', aura_type='professional-ds'):
    api_endpoint = urljoin(api_base, '/v1/instances')

    instance_details = {}

    print()
    print('api_endpoint:', api_endpoint)
    print()
    request_body = {
        "version": "5",
        "region": region,
        "memory": "16GB",
        "name": instance_name,
        "type": aura_type,
        "tenant_id": tenant_id,
        "cloud_provider": cloud_provider
    }

    print('request body type (before):', type(request_body))
    json_request_body = json.dumps(request_body)
    print('request body type (after):', type(json_request_body))

    curl_cmd = "curl -s -X 'POST' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}' -H 'Content-Type: application/json'".format(api_endpoint, access_token)
    curl_cmd += " -d '{}'".format(json_request_body)
    print('curl_cmd:')
    print(curl_cmd)
    api_response = json.loads(subprocess.check_output(curl_cmd, shell=True))

    print('New instance created:')
    print(api_response)
    
    response_data = api_response['data']
    
    instance_details['id'] = response_data['id']
    instance_details['name'] = response_data['name']
    instance_details['connection_url'] = response_data['connection_url']
    instance_details['password'] = response_data['password']
    instance_details['username'] = response_data['username']
    instance_details['cloud_provider'] = response_data['cloud_provider']
    instance_details['region'] = response_data['region']
    instance_details['tenant_id'] = response_data['tenant_id']
    instance_details['type'] = response_data['type']

    print()
    print('instance_details:')
    for item in instance_details:
        print(item, '=', instance_details[item])
    print()

    return instance_details

def list_instances(access_token, api_base, tenant_id=None):
    api_endpoint = urljoin(api_base, '/v1/instances') 
    aura_instances = {}
    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    list_instances = json.loads(subprocess.check_output(list_cmd, shell=True))['data']
    for instance in list_instances:
        aura_instances[instance['id']] = {}
        aura_instances[instance['id']]['instance_id'] = instance['id'] 
        aura_instances[instance['id']]['instance_name'] = instance['name'] 
        aura_instances[instance['id']]['cloud_provider'] = instance['cloud_provider'] 
        aura_instances[instance['id']]['tenant_id'] = instance['tenant_id'] 

    return aura_instances

def get_instance_info(access_token, api_base, instance_id):
    api_endpoint = urljoin(api_base, '/v1/instances/' + instance_id)
    instance_data = {}

    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    api_response = json.loads(subprocess.check_output(info_cmd, shell=True))

    if 'data' in api_response:
        response_data = api_response['data']
        instance_status = response_data['status']

    return response_data

Now we can deploy our first instance!

In [ ]:
instance_name = "my_new_aura_instance"
cloud_provider = 'gcp'
instance_info = deploy_instance(access_token, api_base, instance_name, tenant_id, cloud_provider)
print('instance_info:')
print(instance_info)

Let's take a closer look at one of the instances.

In [ ]:
instance_details = get_instance_info(access_token, api_base, instance_info['id'])
print('instance_details:')
print()
for item in instance_details:
    print('\t' + item, '=', instance_details[item])